In [ ]:
#!pip install boto3

   ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.1 MB 3.4 MB/s eta 0:00:04
   --- ------------------------------------ 1.3/13.1 MB 3.7 MB/s eta 0:00:04
   ------ --------------------------------- 2.1/13.1 MB 3.6 MB/s eta 0:00:04
   -------- ------------------------------- 2.9/13.1 MB 3.6 MB/s eta 0:00:03
   ----------- ---------------------------- 3.7/13.1 MB 3.5 MB/s eta 0:00:03
   ------------- -------------------------- 4.5/13.1 MB 3.5 MB/s eta 0:00:03
   --------------- ------------------------ 5.2/13.1 MB 3.6 MB/s eta 0:00:03
   ------------------ --------------------- 6.0/13.1 MB 3.6 MB/s eta 0:00:02
   --------------------- ------------------ 7.1/13.1 MB 3.7 MB/s eta 0:00:02
   ---------------------- ----------------- 7.3/13.1 MB 3.7 MB/s eta 0:00:02
   ------------------------ --------------- 8.1/13.1 MB 3.5 MB/s eta 0:00:02
   --------------------------- ------------ 8.9/13.1 MB 3.5 MB/s eta 0:00:02
   ---

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.35.73 which is incompatible.


In [ ]:
#!pip install sagemaker

  Using cached sagemaker-2.235.2-py3-none-any.whl.metadata (16 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached importlib_metadata-6.11.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached pathos-0.3.3-py3-none-any.whl.metadata (11 kB)
  Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached sagemaker_core-1.0.16-py3-none-any.whl.metadata (4.9 kB)
  Using cached schema-0.7.7-py2.py3-none-any.whl.metadata (34 kB)
  Using cached smdebug_rulesconfig-1.0.1-py2.py3-none-any.whl.metadata (943 bytes)
  Using cached platformdirs-4.3.6-py3-none-any.whl.metadata (11 kB)
  Using cached mock-4.0.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached ppft-1.7.6.9-py3-none-any.whl.metadata (12 kB)
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached pox-0.3.5-py3-none-any.whl.metadata (8.0 kB)
  Using cached multiprocess-0.70.17-py312-none-any.whl.metadata (7.2 kB)
Using cached sagemaker-2.235.2-py3-none-any.w

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.114 requires urllib3>=2.2.2, but you have urllib3 2.0.7 which is incompatible.
dask 2024.8.2 requires cloudpickle>=3.0.0, but you have cloudpickle 2.2.1 which is incompatible.
distributed 2024.8.2 requires cloudpickle>=3.0.0, but you have cloudpickle 2.2.1 which is incompatible.
opentelemetry-proto 1.28.2 requires protobuf<6.0,>=5.0, but you have protobuf 4.25.5 which is incompatible.


In [10]:

import boto3
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os

c:\Users\mindf\anaconda3\Lib\site-packages\pydantic\_internal\_fields.py:201: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[12/03/24 23:18:59] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=447963;file://c:\Users\mindf\anaconda3\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=514712;file://c:\Users\mindf\anaconda3\Lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\mindf\AppData\Local\sagemaker\sagemaker\config.yaml


In [21]:
# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
#role = sagemaker.get_execution_role()
role = r"arn:aws:iam::069641793383:role/service-role/AmazonSageMaker-ExecutionRole-20241128T231146"
bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/california-housing"

[12/03/24 23:33:39] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=895512;file://c:\Users\mindf\anaconda3\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=22502;file://c:\Users\mindf\anaconda3\Lib\site-packages\botocore\credentials.py#1278\1278]8;;\

In [22]:
# Load California Housing dataset
california_data = fetch_california_housing(as_frame=True)
X = california_data.data
y = california_data.target

In [23]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
# Save training and test data as CSV
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

In [25]:
train_data_path = "train.csv"
test_data_path = "test.csv"
train_data.to_csv(train_data_path, index=False)
test_data.to_csv(test_data_path, index=False)

In [26]:
# Upload data to S3
train_data_s3_path = sagemaker_session.upload_data(train_data_path, bucket=bucket, key_prefix=prefix)
test_data_s3_path = sagemaker_session.upload_data(test_data_path, bucket=bucket, key_prefix=prefix)

In [27]:
print(f"Training data uploaded to: {train_data_s3_path}")
print(f"Test data uploaded to: {test_data_s3_path}")

Training data uploaded to: s3://sagemaker-ap-south-1-069641793383/sagemaker/california-housing/train.csv
Test data uploaded to: s3://sagemaker-ap-south-1-069641793383/sagemaker/california-housing/test.csv


In [28]:
# Create SKLearn Estimator
script_path = "training-script.py" 

In [33]:
output_path = f"s3://sagemaker-ap-south-1-069641793383/sagemaker/output"
sklearn_estimator = SKLearn(
    entry_point=script_path,
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="1.2-1",
    sagemaker_session=sagemaker_session,
    output_path=output_path,
    hyperparameters={"max_depth": 10},
)

In [34]:
# Start training
sklearn_estimator.fit({"train": train_data_s3_path})

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\mindf\anaconda3\Lib\pathlib.py:860 in exists                                            │
│                                                                                                  │
│    857 │   │   add the argument follow_symlinks=False.                                           │
│    858 │   │   """                                                                               │
│    859 │   │   try:                                                                              │
│ ❱  860 │   │   │   self.stat(follow_symlinks=follow_symlinks)                                    │
│    861 │   │   except OSError as e:                                                              │
│    862 │   │   │   if not _ignore_error(e):                                                      │
│    863 │   │   │   │   raise                                                                     │
│                                                                                                  │
│ c:\Users\mindf\anaconda3\Lib\pathlib.py:840 in stat                                              │
│                                                                                                  │
│    837 │   │   Return the result of the stat() system call on this path, like                    │
│    838 │   │   os.stat() does.                                                                   │
│    839 │   │   """                                                                               │
│ ❱  840 │   │   return os.stat(self, follow_symlinks=follow_symlinks)                             │
│    841 │                                                                                         │
│    842 │   def lstat(self):                                                                      │
│    843 │   │   """                                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [WinError 2] The system cannot find the file specified: 'c:\\.sagemaker-code-config'

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Start training                                                                             │
│ ❱ 2 sklearn_estimator.fit({"train": train_data_s3_path})                                         │
│   3                                                                                              │
│                                                                                                  │
│ c:\Users\mindf\anaconda3\Lib\site-packages\sagemaker\workflow\pipeline_context.py:346 in wrapper │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                        

In [ ]:
# Deploy model as endpoint
predictor = sklearn_estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="california-housing-endpoint-2"
)

In [ ]:
# Test predictions
test_sample = X_test.iloc[0:1]
response = predictor.predict(test_sample.to_numpy())
print(f"Prediction: {response}")

In [ ]:
import boto3

sagemaker_client = boto3.client("sagemaker")

# Delete the existing endpoint configuration
sagemaker_client.delete_endpoint_config(EndpointConfigName="california-housing-endpoint")
